In [2]:
import datetime
import json
from bs4 import BeautifulSoup
import requests
from dataclasses import dataclass
import types

@dataclass
class Holiday:
    name: str
    date: datetime
    
    def __str__ (self):
        return f"{name} ({date})"

In [3]:
class HolidayList:
    def __init__(self):
        self.innerHolidays = []

    def addHoliday(holidayObj):
        if isinstance(holidayObj, types.ListType):
            innerHolidays.append(holidayObj)
            return(f"Added holiday {holidayObj}")


In [22]:
import requests

url = "https://community-open-weather-map.p.rapidapi.com/forecast/daily"

querystring = {"q":"new york,us","lat":"41","lon":"74","cnt":"7","units":"imperial"}

headers = {
    'x-rapidapi-host': "community-open-weather-map.p.rapidapi.com",
    'x-rapidapi-key': "1e19830f3bmshf56abb61a6e9168p189818jsnf035a12afc26"
    }

weather = requests.request("GET", url, headers=headers, params=querystring)
weather_dict = weather.json()

print(weather_dict)

print(weather_dict["list"][0]["weather"][0]["main"])

{'city': {'id': 5128581, 'name': 'New York', 'coord': {'lon': -74.006, 'lat': 40.7143}, 'country': 'US', 'population': 8175133, 'timezone': -18000}, 'cod': '200', 'message': 0.2635796, 'cnt': 7, 'list': [{'dt': 1641574800, 'sunrise': 1641557998, 'sunset': 1641591856, 'temp': {'day': 32.88, 'min': 24.64, 'max': 35.49, 'night': 24.64, 'eve': 30.06, 'morn': 29.59}, 'feels_like': {'day': 22.14, 'night': 12.04, 'eve': 18.36, 'morn': 21.4}, 'pressure': 1011, 'humidity': 71, 'weather': [{'id': 601, 'main': 'Snow', 'description': 'snow', 'icon': '13d'}], 'speed': 17.65, 'deg': 304, 'gust': 31.32, 'clouds': 83, 'pop': 1, 'snow': 9.67}, {'dt': 1641661200, 'sunrise': 1641644389, 'sunset': 1641678315, 'temp': {'day': 27.18, 'min': 21.24, 'max': 30.07, 'night': 27.95, 'eve': 28.27, 'morn': 21.33}, 'feels_like': {'day': 19.6, 'night': 18.82, 'eve': 21.45, 'morn': 9.1}, 'pressure': 1035, 'humidity': 52, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'sky is clear', 'icon': '01d'}], 'speed': 